<a href="https://colab.research.google.com/github/GOURAVRISHI/NLP_lectures/blob/main/Copy_of_text_generation_21may.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment
Use DVC or MLFlow to convert this into 
1. Create and API which will take the first input and it'll generate the next sentence 
or
2. Create and UI for this app

In [2]:
import os
ROOT= "/content/drive/MyDrive/Colab Notebooks/text_generation"
os.chdir(ROOT)

# Text Generation with RNN

In [3]:
import tensorflow as tf
import numpy as np

import time

In [4]:
os.makedirs("data", exist_ok= True)

In [6]:
# !curl https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt > "./data/shakespeare.txt"

In [10]:
# same thing can be done using Config.yaml, also use constructor, 
# view-source:https://en.wikipedia.org/wiki/Main_Page - data link/ copy of page source of html page

class Config:
  path_to_file = os.path.join("data", "input.html")
  seq_length = 100

  Batch_size= 64
  Buffer_size = 10000 # keep things ready for training in buffer

  embedding_dim= 256

  rnn_units= 1024

  EPOCHS = 30

  checkpoint_dir= "./training_ckpt"

In [11]:
Config.EPOCHS

30

In [14]:
# Reading first few lines of the text
text = open(Config.path_to_file, "rb").read().decode(encoding= "utf-8")
text[:1000]

'\n<!-- saved from url=(0039)https://en.wikipedia.org/wiki/Main_Page -->\n<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta name="color-scheme" content="light dark"></head><body><div class="line-gutter-backdrop"></div><form autocomplete="off"><label class="line-wrap-control">Line wrap<input type="checkbox" aria-label="Line wrap"></label></form><table><tbody><tr><td class="line-number" value="1"></td><td class="line-content"><span class="html-doctype">&lt;!DOCTYPE html&gt;</span></td></tr><tr><td class="line-number" value="2"></td><td class="line-content"><span class="html-tag">&lt;html <span class="html-attribute-name">class</span>="<span class="html-attribute-value">client-nojs</span>" <span class="html-attribute-name">lang</span>="<span class="html-attribute-value">en</span>" <span class="html-attribute-name">dir</span>="<span class="html-attribute-value">ltr</span>"&gt;</span></td></tr><tr><td class="line-number" value="3"></td><td class="line-cont

In [15]:
with open(Config.path_to_file) as f:
  t= f.read()

In [16]:
t[:100]

'\n<!-- saved from url=(0039)https://en.wikipedia.org/wiki/Main_Page -->\n<html><head><meta http-equiv='

In [17]:
set(t)

{'\t',
 '\n',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '®',
 'à',
 'â',
 'å',
 'ç',
 'ê',
 'ñ',
 'ú',
 'ü',
 'ă',
 'Č',
 'č',
 'ń',
 'ō',
 'ř',
 'š',
 'ų',
 'Ε',
 'ά',
 'η',
 'ι',
 'κ',
 'λ',
 'ν',
 'Б',
 'М',
 'Р',
 'С',
 'У',
 'а',
 'в',
 'г',
 'д',
 'е',
 'и',
 'й',
 'к',
 'л',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'у',
 'х',
 'ъ',
 'ь',
 'ї',
 'ב',
 'י',
 'ע',
 'ר',
 'ת',
 'ا',
 'ب',
 'ة',
 'ر',
 'س',
 'ع',
 'ف',
 'ل',
 'ي',
 'ی',
 'ং',
 'ব',
 'ল',
 'া',
 'ം',
 'മ',
 'യ',
 'ല',
 

In [18]:
sorted(set(t))

['\t',
 '\n',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '®',
 'à',
 'â',
 'å',
 'ç',
 'ê',
 'ñ',
 'ú',
 'ü',
 'ă',
 'Č',
 'č',
 'ń',
 'ō',
 'ř',
 'š',
 'ų',
 'Ε',
 'ά',
 'η',
 'ι',
 'κ',
 'λ',
 'ν',
 'Б',
 'М',
 'Р',
 'С',
 'У',
 'а',
 'в',
 'г',
 'д',
 'е',
 'и',
 'й',
 'к',
 'л',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'у',
 'х',
 'ъ',
 'ь',
 'ї',
 'ב',
 'י',
 'ע',
 'ר',
 'ת',
 'ا',
 'ب',
 'ة',
 'ر',
 'س',
 'ع',
 'ف',
 'ل',
 'ي',
 'ی',
 'ং',
 'ব',
 'ল',
 'া',
 'ം',
 'മ',
 'യ',
 'ല',
 

In [19]:
vocab = sorted(set(t))
len(vocab)

190

In [20]:
char2idx ={uniChar: idx for idx, uniChar in enumerate(vocab)}
char2idx

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '"': 4,
 '#': 5,
 '$': 6,
 '%': 7,
 '&': 8,
 "'": 9,
 '(': 10,
 ')': 11,
 '*': 12,
 '+': 13,
 ',': 14,
 '-': 15,
 '.': 16,
 '/': 17,
 '0': 18,
 '1': 19,
 '2': 20,
 '3': 21,
 '4': 22,
 '5': 23,
 '6': 24,
 '7': 25,
 '8': 26,
 '9': 27,
 ':': 28,
 ';': 29,
 '<': 30,
 '=': 31,
 '>': 32,
 '?': 33,
 '@': 34,
 'A': 35,
 'B': 36,
 'C': 37,
 'D': 38,
 'E': 39,
 'F': 40,
 'G': 41,
 'H': 42,
 'I': 43,
 'J': 44,
 'K': 45,
 'L': 46,
 'M': 47,
 'N': 48,
 'O': 49,
 'P': 50,
 'Q': 51,
 'R': 52,
 'S': 53,
 'T': 54,
 'U': 55,
 'V': 56,
 'W': 57,
 'X': 58,
 'Y': 59,
 'Z': 60,
 '[': 61,
 '\\': 62,
 ']': 63,
 '_': 64,
 'a': 65,
 'b': 66,
 'c': 67,
 'd': 68,
 'e': 69,
 'f': 70,
 'g': 71,
 'h': 72,
 'i': 73,
 'j': 74,
 'k': 75,
 'l': 76,
 'm': 77,
 'n': 78,
 'o': 79,
 'p': 80,
 'q': 81,
 'r': 82,
 's': 83,
 't': 84,
 'u': 85,
 'v': 86,
 'w': 87,
 'x': 88,
 'y': 89,
 'z': 90,
 '{': 91,
 '|': 92,
 '}': 93,
 '®': 94,
 'à': 95,
 'â': 96,
 'å': 97,
 'ç': 98,
 'ê': 99,
 'ñ': 1

In [21]:
# all text is now represented as integer
text_as_int = np.array([char2idx[c] for c in t ])
text_as_int

array([ 1, 30,  3, ..., 77, 76, 32])

In [22]:
t[:13], text_as_int[:13]

('\n<!-- saved f', array([ 1, 30,  3, 15, 15,  2, 83, 65, 86, 69, 68,  2, 70]))

In [23]:
t

'\n<!-- saved from url=(0039)https://en.wikipedia.org/wiki/Main_Page -->\n<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta name="color-scheme" content="light dark"></head><body><div class="line-gutter-backdrop"></div><form autocomplete="off"><label class="line-wrap-control">Line wrap<input type="checkbox" aria-label="Line wrap"></label></form><table><tbody><tr><td class="line-number" value="1"></td><td class="line-content"><span class="html-doctype">&lt;!DOCTYPE html&gt;</span></td></tr><tr><td class="line-number" value="2"></td><td class="line-content"><span class="html-tag">&lt;html <span class="html-attribute-name">class</span>="<span class="html-attribute-value">client-nojs</span>" <span class="html-attribute-name">lang</span>="<span class="html-attribute-value">en</span>" <span class="html-attribute-name">dir</span>="<span class="html-attribute-value">ltr</span>"&gt;</span></td></tr><tr><td class="line-number" value="3"></td><td class="line-cont

In [24]:
len(t)

416762

In [25]:
example_per_epoch= len(t)//(Config.seq_length+1)
example_per_epoch

4126

In [26]:
idx2char_dict ={val: key for key, val in char2idx.items()}
idx2char_dict

{0: '\t',
 1: '\n',
 2: ' ',
 3: '!',
 4: '"',
 5: '#',
 6: '$',
 7: '%',
 8: '&',
 9: "'",
 10: '(',
 11: ')',
 12: '*',
 13: '+',
 14: ',',
 15: '-',
 16: '.',
 17: '/',
 18: '0',
 19: '1',
 20: '2',
 21: '3',
 22: '4',
 23: '5',
 24: '6',
 25: '7',
 26: '8',
 27: '9',
 28: ':',
 29: ';',
 30: '<',
 31: '=',
 32: '>',
 33: '?',
 34: '@',
 35: 'A',
 36: 'B',
 37: 'C',
 38: 'D',
 39: 'E',
 40: 'F',
 41: 'G',
 42: 'H',
 43: 'I',
 44: 'J',
 45: 'K',
 46: 'L',
 47: 'M',
 48: 'N',
 49: 'O',
 50: 'P',
 51: 'Q',
 52: 'R',
 53: 'S',
 54: 'T',
 55: 'U',
 56: 'V',
 57: 'W',
 58: 'X',
 59: 'Y',
 60: 'Z',
 61: '[',
 62: '\\',
 63: ']',
 64: '_',
 65: 'a',
 66: 'b',
 67: 'c',
 68: 'd',
 69: 'e',
 70: 'f',
 71: 'g',
 72: 'h',
 73: 'i',
 74: 'j',
 75: 'k',
 76: 'l',
 77: 'm',
 78: 'n',
 79: 'o',
 80: 'p',
 81: 'q',
 82: 'r',
 83: 's',
 84: 't',
 85: 'u',
 86: 'v',
 87: 'w',
 88: 'x',
 89: 'y',
 90: 'z',
 91: '{',
 92: '|',
 93: '}',
 94: '®',
 95: 'à',
 96: 'â',
 97: 'å',
 98: 'ç',
 99: 'ê',
 100: '

In [27]:
idx2char= np.array(vocab)
idx2char

array(['\t', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*',
       '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7',
       '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D',
       'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q',
       'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_',
       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
       'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
       '{', '|', '}', '®', 'à', 'â', 'å', 'ç', 'ê', 'ñ', 'ú', 'ü', 'ă',
       'Č', 'č', 'ń', 'ō', 'ř', 'š', 'ų', 'Ε', 'ά', 'η', 'ι', 'κ', 'λ',
       'ν', 'Б', 'М', 'Р', 'С', 'У', 'а', 'в', 'г', 'д', 'е', 'и', 'й',
       'к', 'л', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'х', 'ъ', 'ь', 'ї',
       'ב', 'י', 'ע', 'ר', 'ת', 'ا', 'ب', 'ة', 'ر', 'س', 'ع', 'ف', 'ل',
       'ي', 'ی', 'ং', 'ব', 'ল', 'া', 'ം', 'മ', 'യ', 'ല', 'ള', 'ാ', 'ท',
       'ย', 'ไ', 'ა', 'თ', 'ი', 'ლ', 'რ', 'უ', 'ქ', 'ế', 'ệ',

In [28]:
# We can use any one of them

idx2char[0], idx2char_dict[0]

('\t', '\t')

In [29]:
print("Hi\nThere")

Hi
There


In [30]:
# getting exact representation of words

print(r"Hi\nThere")

Hi\nThere


In [31]:
# getting exact representation of words

print(repr("Hi\nThere"))

'Hi\nThere'


In [32]:
# Training example per target 
# every character is passed as a data 

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# get tensors 
for i in char_dataset.take(10):
  print(i)

# get tensors 

tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(30, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(15, shape=(), dtype=int64)
tf.Tensor(15, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(83, shape=(), dtype=int64)
tf.Tensor(65, shape=(), dtype=int64)
tf.Tensor(86, shape=(), dtype=int64)
tf.Tensor(69, shape=(), dtype=int64)


In [33]:
for i in char_dataset.take(13):
  print(idx2char[i.numpy()])

# get tensors 



<
!
-
-
 
s
a
v
e
d
 
f


In [34]:
# Training example per target 
# creating batches
sequences = char_dataset.batch(Config.seq_length+1, drop_remainder= True)

for item in sequences.take(5):
  print(item.numpy())

[ 1 30  3 15 15  2 83 65 86 69 68  2 70 82 79 77  2 85 82 76 31 10 18 18
 21 27 11 72 84 84 80 83 28 17 17 69 78 16 87 73 75 73 80 69 68 73 65 16
 79 82 71 17 87 73 75 73 17 47 65 73 78 64 50 65 71 69  2 15 15 32  1 30
 72 84 77 76 32 30 72 69 65 68 32 30 77 69 84 65  2 72 84 84 80 15 69 81
 85 73 86 31  4]
[37 79 78 84 69 78 84 15 54 89 80 69  4  2 67 79 78 84 69 78 84 31  4 84
 69 88 84 17 72 84 77 76 29  2 67 72 65 82 83 69 84 31 55 54 40 15 26  4
 32 30 77 69 84 65  2 78 65 77 69 31  4 67 79 76 79 82 15 83 67 72 69 77
 69  4  2 67 79 78 84 69 78 84 31  4 76 73 71 72 84  2 68 65 82 75  4 32
 30 17 72 69 65]
[68 32 30 66 79 68 89 32 30 68 73 86  2 67 76 65 83 83 31  4 76 73 78 69
 15 71 85 84 84 69 82 15 66 65 67 75 68 82 79 80  4 32 30 17 68 73 86 32
 30 70 79 82 77  2 65 85 84 79 67 79 77 80 76 69 84 69 31  4 79 70 70  4
 32 30 76 65 66 69 76  2 67 76 65 83 83 31  4 76 73 78 69 15 87 82 65 80
 15 67 79 78 84]
[82 79 76  4 32 46 73 78 69  2 87 82 65 80 30 73 78 80 85 84  2 84 89 80


In [35]:
# Training example per target 
sequences = char_dataset.batch(Config.seq_length+1, drop_remainder= True)

for item in sequences.take(5):
  to_print= repr("".join([idx2char[c] for c in item.numpy()]))
  print(to_print)
  print("length of sentence: ", len(to_print))

'\n<!-- saved from url=(0039)https://en.wikipedia.org/wiki/Main_Page -->\n<html><head><meta http-equiv="'
length of sentence:  105
'Content-Type" content="text/html; charset=UTF-8"><meta name="color-scheme" content="light dark"></hea'
length of sentence:  103
'd><body><div class="line-gutter-backdrop"></div><form autocomplete="off"><label class="line-wrap-cont'
length of sentence:  103
'rol">Line wrap<input type="checkbox" aria-label="Line wrap"></label></form><table><tbody><tr><td clas'
length of sentence:  103
's="line-number" value="1"></td><td class="line-content"><span class="html-doctype">&lt;!DOCTYPE html&'
length of sentence:  103


In [36]:
for item in sequences.take(5):
  print(repr("".join([idx2char[c] for c in item.numpy()])))

'\n<!-- saved from url=(0039)https://en.wikipedia.org/wiki/Main_Page -->\n<html><head><meta http-equiv="'
'Content-Type" content="text/html; charset=UTF-8"><meta name="color-scheme" content="light dark"></hea'
'd><body><div class="line-gutter-backdrop"></div><form autocomplete="off"><label class="line-wrap-cont'
'rol">Line wrap<input type="checkbox" aria-label="Line wrap"></label></form><table><tbody><tr><td clas'
's="line-number" value="1"></td><td class="line-content"><span class="html-doctype">&lt;!DOCTYPE html&'


In [37]:
"".join([idx2char[c] for c in item.numpy()])

's="line-number" value="1"></td><td class="line-content"><span class="html-doctype">&lt;!DOCTYPE html&'

In [38]:
# split input and target
# First Citizen->irst Citizen

In [39]:
def split_input_target(chunk):
  input_text = chunk[:-1] # First Citizen
  target_text= chunk[1:]  # irst Citizen
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [40]:
# trying to predict the next character
for input_example, target_example in dataset.take(1):
  print("input_data:\n")
  print(repr("".join([idx2char[i] for i in input_example.numpy()])))

  print("\ntarget_data:\n")
  print(repr("".join([idx2char[t] for t in target_example.numpy()])))

input_data:

'\n<!-- saved from url=(0039)https://en.wikipedia.org/wiki/Main_Page -->\n<html><head><meta http-equiv='

target_data:

'<!-- saved from url=(0039)https://en.wikipedia.org/wiki/Main_Page -->\n<html><head><meta http-equiv="'


In [41]:
# drop_remainder means length can vary. If shorter length then drop the remaining
dataset= dataset.shuffle(Config.Buffer_size).batch(Config.Batch_size, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [42]:
Config.vocab_size= len(vocab)
Config.vocab_size

190

In [43]:
def build_model(
    vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
          tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape= [batch_size, None]),
          tf.keras.layers.GRU(rnn_units,
                              return_sequences= True,
                              stateful= True,
                              recurrent_initializer="glorot_uniform"),
          tf.keras.layers.Dense(vocab_size)
  ])

  return model

In [44]:
model= build_model(
    vocab_size= Config.vocab_size, 
    embedding_dim = Config.embedding_dim, 
    rnn_units= Config.rnn_units, 
    batch_size= Config.Batch_size
)

In [45]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           48640     
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 190)           194750    
                                                                 
Total params: 4,181,694
Trainable params: 4,181,694
Non-trainable params: 0
_________________________________________________________________


In [46]:
# logits means raw values at the outcome
def loss(labels, logits): 
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits= True)

In [47]:
model.compile(optimizer="adam", loss=loss)

In [48]:
checkpoint_prefix= os.path.join(Config.checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath= checkpoint_prefix, 
    save_weights_only= True   )

In [49]:
history= model.fit(dataset, epochs= Config.EPOCHS, callbacks= [checkpoint_callback])

Epoch 1/30
64/64 [==============================] - 10s 60ms/step - loss: 2.6575
Epoch 2/30
64/64 [==============================] - 4s 60ms/step - loss: 0.9983
Epoch 3/30
64/64 [==============================] - 4s 61ms/step - loss: 0.6641
Epoch 4/30
64/64 [==============================] - 4s 61ms/step - loss: 0.5474
Epoch 5/30
64/64 [==============================] - 4s 62ms/step - loss: 0.4764
Epoch 6/30
64/64 [==============================] - 4s 61ms/step - loss: 0.4231
Epoch 7/30
64/64 [==============================] - 4s 62ms/step - loss: 0.3808
Epoch 8/30
64/64 [==============================] - 5s 63ms/step - loss: 0.3458
Epoch 9/30
64/64 [==============================] - 4s 63ms/step - loss: 0.3179
Epoch 10/30
64/64 [==============================] - 5s 63ms/step - loss: 0.2904
Epoch 11/30
64/64 [==============================] - 5s 63ms/step - loss: 0.2710
Epoch 12/30
64/64 [==============================] - 4s 62ms/step - loss: 0.2537
Epoch 13/30
64/64 [=================

In [50]:
(len(text)/Config.Batch_size)/(Config.seq_length+1)

64.4743193069307

In [51]:
# Restoring checkpoint

tf.train.latest_checkpoint(Config.checkpoint_dir)

'./training_ckpt/ckpt_30'

In [52]:
Config.vocab_size= len(vocab)
Config.vocab_size

190

In [53]:
model_from_ckpt= build_model(
    vocab_size= Config.vocab_size, 
    embedding_dim = Config.embedding_dim, 
    rnn_units= Config.rnn_units, 
    batch_size= 1
)

In [54]:
model_from_ckpt.load_weights(tf.train.latest_checkpoint(Config.checkpoint_dir))

model_from_ckpt.build(tf.TensorShape([1, None]))


In [55]:
model_from_ckpt.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            48640     
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 190)            194750    
                                                                 
Total params: 4,181,694
Trainable params: 4,181,694
Non-trainable params: 0
_________________________________________________________________


# Prediction

In [56]:
# define a 

def generate_text(model, start_string, no_of_chars_to_gen= 1000):
  # convert input text to no.
  input_val =[char2idx[s] for s in start_string]  # text converted into int
  input_val = tf.expand_dims(input_val, 0)         # []  ->> [1,]

  text_generated= list()

  temperature = 1.0

  model.reset_states()  # resetting the previous states if any while predictions.

  for i in range(no_of_chars_to_gen):
    predictions = model(input_val)
    
    predictions= tf.squeeze(predictions, 0)

    predictions= predictions/temperature
    print(predictions)
    predicted_id = tf.random.categorical(predictions, num_samples= 1)[-1,0].numpy() # using numpy will give int value
    print(predicted_id)

    input_val= tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])

  return start_string +"".join(text_generated)

In [62]:
# , no_of_chars_to_gen= 10
result = generate_text(model=model_from_ckpt, start_string= "<h3>test</h3>")
print(result)

Streaming output truncated to the last 5000 lines.
   -4.68666     -4.5339823   -5.5749254   -3.9719477    0.16855204
   -0.7137511   -2.6007838    2.625792    -1.817366     2.122563
    2.2132466    1.2238443    2.7548807    0.380159    -0.8930057
   -2.0118847   -2.1104813   -1.253        1.5179626   -0.24246791
    5.6325536   -1.71192      6.147838    -8.353253    -0.0857505
   -4.3700075   -7.0121417   -5.2736473   -5.8309584   -4.443998
   -5.2533054   -7.892931    -9.276622    -5.8172655   -2.2049215
   -7.9043803   -3.916709    -3.2169433   -5.3384767   -2.1134782
   -6.1719403   -3.3700309   -5.13727     -5.780089    -7.3475113
   -6.3033934   -6.260457    -4.3578315   -5.891825    -6.106921
   -7.682156   -10.820686    -5.5135274   -2.4630413    5.013832
   -1.6458601   -2.882134     1.8041968    0.98146784   0.21518861
   -2.2935038   -0.05719015   9.032473    -1.811171    -5.4846206
   -5.6989684   14.547856     0.15404727   4.941639    -0.37999326
    1.6004151   -4.039305

In [64]:
# !pip install HTMLrenderer

In [66]:
from HTMLrenderer.render import render_HTML
render_HTML(result)

""


In [58]:
tf.math.log([[0.5 ,0.5]])

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-0.6931472, -0.6931472]], dtype=float32)>

In [59]:
samples = tf.random.categorical([[1.,2.,3.,4.]], 1)
samples.numpy()

array([[3]])

In [60]:
for _ in range(10):
  values = [[1., 2., 3., 4.]]
  samples = tf.random.categorical(values, 1)[-1, 0]

  print(samples.numpy())

3
3
3
3
2
3
3
1
3
3


In [61]:
for _ in range(10):
  predictons = [[1., 2., 3., 4.,55., 56., 100., 101., 200.]]
  samples = tf.random.categorical(values, 1)[-1, 0]

  print(samples.numpy())

0
3
3
2
3
3
2
0
1
3
